In [1]:
# dependencies
import pandas as pd
import os

In [2]:
# set file paths
schoolsPath = os.path.join("Resources", "schools_complete.csv")
studentsPath = os.path.join("Resources", "students_complete.csv")

# read in csv files
schools = pd.read_csv(schoolsPath)
students = pd.read_csv(studentsPath)

In [3]:
# combine both files into one csv dataset
schoolNstudents = pd.merge(students, schools, how="left", on=["school_name", "school_name"])
schoolNstudents

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [4]:
# calculate the total number of schools
unique_schools = schoolNstudents["school_name"].unique()
total_schools = len(unique_schools)
total_schools

15

In [5]:
# calculate the total number of students
total_students = len(schoolNstudents["student_name"])
total_students

39170

In [6]:
# calculate the total budget
total_budget = schools["budget"].sum()
total_budget

24649428

In [7]:
# calculate the average math score
average_math = students["math_score"].mean()
average_math

78.98537145774827

In [8]:
# calculate the average reading score
average_reading = students["reading_score"].mean()
average_reading

81.87784018381414

In [9]:
# calculate the percentage of students with a passing math score (70 or greater)
math_greater_70 = students.loc[students["math_score"] >= 70, :]
math_passing_percent = (len(math_greater_70) / total_students)*100
math_passing_percent

74.9808526933878

In [10]:
# calculate the percentage of students with a passing reading score (70 or greater)
reading_greater_70 = students.loc[students["reading_score"] >= 70, :]
reading_passing_percent = (len(reading_greater_70) / total_students)*100
reading_passing_percent

85.80546336482001

In [11]:
# calculate the percentage of students who passed math and reading (% Overall Passing)
passing_math_reading = students.loc[(students["math_score"] >= 70) & (students["reading_score"] >= 70), :]
overall_passing_percent = (len(passing_math_reading) / total_students)*100
overall_passing_percent

65.17232575950983

In [12]:
# Create a dataframe to hold the above results
district_summary = pd.DataFrame({"Total Schools": [total_schools],
                        "Total Students": [total_students],
                        "Total Budget": [total_budget],
                        "Average Math Score": [average_math],
                        "Average Reading Score": [average_reading],
                        "% Passing Math": [math_passing_percent],
                        "% Passing Reading": [reading_passing_percent],
                        "% Overall Passing": [overall_passing_percent],
                        })

In [13]:
# converting integers to float for formatting
district_summary["Total Students"] = district_summary["Total Students"].astype(float)
district_summary["Total Budget"] = district_summary["Total Budget"].astype(float)

In [14]:
# formatting for better read of district summary
district_summary["Total Students"] = district_summary["Total Students"].map('{:,.0f}'.format)
district_summary["Total Budget"] = district_summary["Total Budget"].map('${:,.2f}'.format)
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [15]:
school_summary_df = schools
school_summary_df["per_student_budget"] = ((school_summary_df["budget"] / school_summary_df["size"]).map('${:,.2f}'.format))

In [16]:
math_per_school = students.groupby("school_name") ["math_score"].sum()
student_per_school = students.groupby("school_name") ["math_score"].count()
average_math_score = math_per_school / student_per_school
#average_math_score

In [17]:
reading_per_school = students.groupby("school_name") ["reading_score"].sum()
average_reading_score = reading_per_school / student_per_school
#average_reading_score

In [31]:
# calculate the percentage of students with a passing math score (70 or greater) per school
math_greater_70_per_school = students.loc[students["math_score"] >= 70, :]
group_math_greater_70_per_school = math_greater_70_per_school.groupby("school_name") ["math_score"].count()
math_percent_passing_per_school = (group_math_greater_70_per_school / student_per_school) * 100
math_percent_passing_per_school

school_name
Bailey High School       66.680064
Cabrera High School      94.133477
Figueroa High School     65.988471
Ford High School         68.309602
Griffin High School      93.392371
Hernandez High School    66.752967
Holden High School       92.505855
Huang High School        65.683922
Johnson High School      66.057551
Pena High School         94.594595
Rodriguez High School    66.366592
Shelton High School      93.867121
Thomas High School       93.272171
Wilson High School       93.867718
Wright High School       93.333333
Name: math_score, dtype: float64

In [32]:
# calculate the percentage of students with a passing reading score (70 or greater) per school
reading_greater_70_per_school = students.loc[students["reading_score"] >= 70, :]
group_reading_greater_70_per_school = reading_greater_70_per_school.groupby("school_name") ["reading_score"].count()
reading_percent_passing_per_school = (group_reading_greater_70_per_school / student_per_school) * 100
reading_percent_passing_per_school

school_name
Bailey High School       81.933280
Cabrera High School      97.039828
Figueroa High School     80.739234
Ford High School         79.299014
Griffin High School      97.138965
Hernandez High School    80.862999
Holden High School       96.252927
Huang High School        81.316421
Johnson High School      81.222432
Pena High School         95.945946
Rodriguez High School    80.220055
Shelton High School      95.854628
Thomas High School       97.308869
Wilson High School       96.539641
Wright High School       96.611111
dtype: float64

In [36]:
# calculate the percentage of students who passed math and reading (% Overall Passing) per school
passing_math_reading_per_school = students.loc[(students["math_score"] >= 70) & (students["reading_score"] >= 70), :]
group_passing_math_reading_per_school = passing_math_reading_per_school.groupby("school_name") ["school_name"].count()
overall_passing_percent_per_school = (passing_math_reading_per_school / student_per_school)*100
overall_passing_percent_per_school

TypeError: unsupported operand type(s) for /: 'str' and 'float'